In [1]:
import geopandas as gpd

# Chemin vers le .shp (prend le nom du shapefile dans ton dossier)
shp_path = "../data/extracted/Shipping_Accidents/Shipping_Accidents.shp"  # adapte le chemin si besoin

# Lecture du shapefile avec geopandas
gdf = gpd.read_file(shp_path)

# Si tu veux bosser comme un DataFrame normal
df = gdf.drop(columns='geometry')  # ou gdf.to_pandas() si tu veux perdre la colonne géométrique

print("Colonnes :", df.columns.tolist())
print(df)

Colonnes : ['Unique_ID', 'Country', 'Year', 'Date', 'Time', 'Location', 'Acc_Type', 'Colli_Type', 'Acc_Detail', 'CauseDetai', 'Assistance', 'Offence', 'Damage', 'HumanEleme', 'IceCondit', 'CrewIceTra', 'Pollution', 'Pollu_m3', 'Pollu_t', 'Pollu_Type', 'RespAction', 'Cargo_Type', 'Ship1_Name', 'Sh1_Categ', 'Sh1_Type', 'Sh1_Hull', 'Sh1Size_gt', 'Sh1Sizedwt', 'Sh1Draug_m', 'Cause_Sh1', 'Pilot_Sh1', 'Ship2_Name', 'Sh2_Categ', 'Sh2_Type', 'Sh2_Hull', 'Sh2Size_gt', 'Sh2Sizedwt', 'Sh2Draug_m', 'Cause_Sh2', 'Pilot_Sh2', 'Add_Info', 'Latitude', 'Longitude', 'source']
      Unique_ID  Country    Year       Date         Time Location   Acc_Type  \
0     1989_0001     None  1989.0 1989-01-07         None     None  Collision   
1     1989_0002     None  1989.0 1989-04-01         None     None  Grounding   
2     1989_0003     None  1989.0 1989-05-01         None     None  Grounding   
3     1989_0004     None  1989.0 1989-05-18         None     None  Collision   
4     1989_0005     None  1989.0 19

In [2]:
# Enregistrer la version nettoyée 
df.to_csv("../data/cleaned/shipping_accidents_cleaned.csv", index=False)
print("Fichier enregistré : /data/cleaned/shipping_accidents_cleaned.csv")

Fichier enregistré : /data/cleaned/shipping_accidents_cleaned.csv
